In [1]:
import pandas as pd

def read_checkpoint_nacional():
    votaciones = pd.read_csv('./out/checkpoint_nacional.csv')
    return votaciones

# Get the data
df = read_checkpoint_nacional()

In [4]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import ipywidgets as widgets
from IPython.display import display


# Mostrar las primeras filas para verificar
print("Estructura de los datos:")
display(df.head())

# Información básica del dataset
print("\nInformación del dataset:")
display(df.info())

# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
display(df.describe())

Estructura de los datos:


,IDEOLOGIA,resultado,GOOGLE_MENSUAL,GOOGLE_SEMANAL,encuesta,nombre_encuesta,año
0,Centro derecha,5.510000,4.866667,5.216667,3.233333,Encuesta 1,2014
1,Centro izquierda,12.056667,11.243333,9.176667,12.633333,Encuesta 1,2014
2,Derecha,10.370000,8.063333,9.036667,9.833333,Encuesta 1,2014
3,Izquierda,5.400000,9.153333,9.893333,3.366667,Encuesta 1,2014
4,Centro derecha,5.510000,4.866667,5.216667,4.800000,Gallup,2014



Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   IDEOLOGIA        36 non-null     object 
 1   resultado        36 non-null     float64
 2   GOOGLE_MENSUAL   36 non-null     float64
 3   GOOGLE_SEMANAL   36 non-null     float64
 4   encuesta         36 non-null     float64
 5   nombre_encuesta  36 non-null     object 
 6   año              36 non-null     int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 2.1+ KB


None


Estadísticas descriptivas:


,resultado,GOOGLE_MENSUAL,GOOGLE_SEMANAL,encuesta,año
count,36.000000,36.000000,36.000000,36.000000,36.000000
mean,15.278056,15.263472,15.266944,14.109722,2018.000000
std,11.330283,12.935843,12.592740,10.835091,3.312315
min,3.740000,4.695000,4.950000,3.133333,2014.000000
25%,5.482500,8.090833,8.625000,4.950000,2014.000000
50%,12.455833,10.092500,9.535000,11.600000,2018.000000
75%,21.320000,14.963750,14.836250,20.387500,2022.000000
max,41.750000,49.280000,44.110000,41.000000,2022.000000


In [5]:
df_modelo = df.drop(['nombre_encuesta', 'año'], axis=1)
df_modelo.head()

,IDEOLOGIA,resultado,GOOGLE_MENSUAL,GOOGLE_SEMANAL,encuesta
0,Centro derecha,5.510000,4.866667,5.216667,3.233333
1,Centro izquierda,12.056667,11.243333,9.176667,12.633333
2,Derecha,10.370000,8.063333,9.036667,9.833333
3,Izquierda,5.400000,9.153333,9.893333,3.366667
4,Centro derecha,5.510000,4.866667,5.216667,4.800000


In [6]:
# Especificar manualmente qué variables son categóricas
categorical_features = ['IDEOLOGIA']
numerical_features = ['encuesta', 'GOOGLE_MENSUAL','GOOGLE_SEMANAL']

# Verificar los tipos de datos actuales
print("\nTipos de datos actuales:")
display(df_modelo[categorical_features + numerical_features].dtypes)


Tipos de datos actuales:


IDEOLOGIA          object
encuesta          float64
GOOGLE_MENSUAL    float64
GOOGLE_SEMANAL    float64
dtype: object

In [7]:
# Convertir explícitamente las variables categóricas al tipo 'category'
for feature in categorical_features:
    df_modelo[feature] = df_modelo[feature].astype('category')

print("\nTipos de datos después de la conversión:")
display(df_modelo[categorical_features + numerical_features].dtypes)


Tipos de datos después de la conversión:


IDEOLOGIA         category
encuesta           float64
GOOGLE_MENSUAL     float64
GOOGLE_SEMANAL     float64
dtype: object

In [8]:
# Crear copia del DataFrame para preservar los datos originales
df_processed = df_modelo.copy()

# Codificación One-Hot para las variables categóricas
df_encoded = pd.get_dummies(df_processed, 
                           columns=categorical_features, 
                           prefix=categorical_features)

# Mostrar el resultado de la codificación
print("Primeras filas del DataFrame con codificación One-Hot:")
display(df_encoded.head())

# Verificar las nuevas columnas creadas
print("\nColumnas después de la codificación:")
print(df_encoded.columns.tolist())

# Verificar dimensiones
print(f"\nDimensiones originales: {df_modelo.shape}")
print(f"Dimensiones después de codificación: {df_encoded.shape}")

Primeras filas del DataFrame con codificación One-Hot:


,resultado,GOOGLE_MENSUAL,GOOGLE_SEMANAL,encuesta,IDEOLOGIA_Centro derecha,IDEOLOGIA_Centro izquierda,IDEOLOGIA_Derecha,IDEOLOGIA_Izquierda
0,5.510000,4.866667,5.216667,3.233333,True,False,False,False
1,12.056667,11.243333,9.176667,12.633333,False,True,False,False
2,10.370000,8.063333,9.036667,9.833333,False,False,True,False
3,5.400000,9.153333,9.893333,3.366667,False,False,False,True
4,5.510000,4.866667,5.216667,4.800000,True,False,False,False



Columnas después de la codificación:
['resultado', 'GOOGLE_MENSUAL', 'GOOGLE_SEMANAL', 'encuesta', 'IDEOLOGIA_Centro derecha', 'IDEOLOGIA_Centro izquierda', 'IDEOLOGIA_Derecha', 'IDEOLOGIA_Izquierda']

Dimensiones originales: (36, 5)
Dimensiones después de codificación: (36, 8)


In [9]:
# Separar características (X) y variable objetivo (y)
X = df_encoded.drop(columns=['resultado'])
y = df_encoded[['resultado']]

# Identificar columnas numéricas para escalar
numerical_columns = ['encuesta', 'GOOGLE_MENSUAL', 'GOOGLE_SEMANAL']

# Crear un escalador y aplicarlo solo a las columnas numéricas
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Normalizar variables numéricas
X_scaled = X.copy()
X_scaled[numerical_columns] = scaler.fit_transform(X_scaled[numerical_columns])

# Mostrar resultado de la normalización
print("Variables independientes (X) después de la normalización:")
display(X_scaled.head())
print("\nEstadísticas de las variables numéricas normalizadas:")
display(X_scaled[numerical_columns].describe())

# Ahora dividir los datos en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, 
    test_size=0.2,     # 20% para prueba, 80% para entrenamiento
    random_state=42    # Semilla para reproducibilidad
)

# Mostrar las dimensiones de los conjuntos de datos
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


Variables independientes (X) después de la normalización:


,GOOGLE_MENSUAL,GOOGLE_SEMANAL,encuesta,IDEOLOGIA_Centro derecha,IDEOLOGIA_Centro izquierda,IDEOLOGIA_Derecha,IDEOLOGIA_Izquierda
0,-0.815122,-0.809422,-1.018051,True,False,False,False
1,-0.315184,-0.490494,-0.138193,False,True,False,False
2,-0.564499,-0.501770,-0.400278,False,False,True,False
3,-0.479042,-0.432776,-1.005570,False,False,False,True
4,-0.815122,-0.809422,-0.871408,True,False,False,False



Estadísticas de las variables numéricas normalizadas:


,encuesta,GOOGLE_MENSUAL,GOOGLE_SEMANAL
count,3.600000e+01,3.600000e+01,3.600000e+01
mean,-1.973730e-16,-1.233581e-17,3.207311e-16
std,1.014185e+00,1.014185e+00,1.014185e+00
min,-1.027411e+00,-8.285805e-01,-8.308987e-01
25%,-8.573674e-01,-5.623432e-01,-5.349242e-01
50%,-2.349148e-01,-4.054102e-01,-4.616353e-01
75%,5.876119e-01,-2.349857e-02,-3.468696e-02
max,2.516981e+00,2.666935e+00,2.322941e+00


X_train shape: (28, 7)
X_test shape: (8, 7)
y_train shape: (28, 1)
y_test shape: (8, 1)


In [10]:
import mlflow
mlflow.set_tracking_uri("/home/janux/Documents/ML VOTACIONES/code/Nacional1/experimentos")
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: '/home/janux/Documents/ML VOTACIONES/code/Nacional1/experimentos'


In [13]:
def optimizar_xgboost(X_train, y_train, X_test, y_test, cv=7):
    """
    Optimiza los hiperparámetros del modelo XGBoost utilizando GridSearchCV.
    
    Parámetros:
    -----------
    X_train : pandas.DataFrame
        Características de entrenamiento
    y_train : pandas.DataFrame o Series
        Variable objetivo de entrenamiento
    X_test : pandas.DataFrame
        Características de prueba
    y_test : pandas.DataFrame o Series
        Variable objetivo de prueba
    cv : int, opcional
        Número de pliegues para la validación cruzada
        
    Retorna:
    --------
    tuple
        (mejor_modelo, mejores_parametros, resultados)
    """
    from sklearn.model_selection import GridSearchCV
    from xgboost import XGBRegressor
    from sklearn.metrics import mean_squared_error, r2_score
    import numpy as np
    import pandas as pd
    import time
    
    print("Iniciando búsqueda de hiperparámetros óptimos para XGBoost...")
    
    # Parámetros a probar
    param_grid = {
        'n_estimators': [25, 50, 100, 200],
        'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
        'max_depth': [3, 5, 7, 9],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
        'gamma': [0, 0.1, 0.2, 0.3]
    }
    
    # Inicializar el modelo base
    xgb_model = XGBRegressor(random_state=42)
    
    # Configurar la búsqueda de cuadrícula
    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        scoring='neg_mean_squared_error',  # Minimiza el error cuadrático medio
        cv=cv,
        n_jobs=-1,  # Usa todos los núcleos disponibles
        verbose=1
    )
    
    # Tiempo de inicio
    start_time = time.time()
    
    # Ajustar la búsqueda de cuadrícula
    grid_search.fit(X_train, y_train)
    
    # Tiempo de finalización
    end_time = time.time()
    tiempo_ejecucion = end_time - start_time
    
    # Obtener los mejores parámetros
    mejores_parametros = grid_search.best_params_
    mejor_puntuacion = -grid_search.best_score_  # Convertir de negativo a positivo
    
    print(f"\nBúsqueda completada en {tiempo_ejecucion:.2f} segundos")
    print(f"Mejores parámetros encontrados: {mejores_parametros}")
    print(f"Mejor MSE durante la validación cruzada: {mejor_puntuacion:.4f}")
    
    # Entrenar el modelo final con los mejores parámetros
    mejor_modelo = XGBRegressor(**mejores_parametros, random_state=42)
    mejor_modelo.fit(X_train, y_train)
    
    # Evaluar en el conjunto de prueba
    y_pred = mejor_modelo.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print("\nEvaluación en conjunto de prueba:")
    print(f"Error cuadrático medio (MSE): {mse:.4f}")
    print(f"Raíz del error cuadrático medio (RMSE): {rmse:.4f}")
    print(f"Coeficiente de determinación (R²): {r2:.4f}")
    
    # Importancia de características
    importancia = mejor_modelo.feature_importances_
    nombres_caracteristicas = X_train.columns
    
    feature_importance = pd.DataFrame({
        'Feature': nombres_caracteristicas,
        'Importance': importancia
    }).sort_values(by='Importance', ascending=False)
    
    print("\nImportancia de características del modelo optimizado:")
    print(feature_importance)
    
    # Resultados
    resultados = {
        'modelo_optimizado': {
            'mse': mse,
            'rmse': rmse,
            'r2': r2,
            'parametros': mejores_parametros
        },
        'tiempo_ejecucion': tiempo_ejecucion,
        'feature_importance': feature_importance
    }
    
    return mejor_modelo, mejores_parametros, resultados

In [14]:
# Aquí va la llamada a la función
mejor_modelo, mejores_params, resultados = optimizar_xgboost(
    X_train, y_train, X_test, y_test
)

# El modelo optimizado
modelo_exp_7_nacional = mejor_modelo

Iniciando búsqueda de hiperparámetros óptimos para XGBoost...
Fitting 7 folds for each of 21504 candidates, totalling 150528 fits

Búsqueda completada en 373.70 segundos
Mejores parámetros encontrados: {'colsample_bytree': 1.0, 'gamma': 0.3, 'learning_rate': 0.5, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 25, 'subsample': 0.9}
Mejor MSE durante la validación cruzada: 10.4428

Evaluación en conjunto de prueba:
Error cuadrático medio (MSE): 3.6686
Raíz del error cuadrático medio (RMSE): 1.9154
Coeficiente de determinación (R²): 0.9817

Importancia de características del modelo optimizado:
                      Feature  Importance
2                    encuesta    0.651851
0              GOOGLE_MENSUAL    0.326830
1              GOOGLE_SEMANAL    0.017592
3    IDEOLOGIA_Centro derecha    0.003727
4  IDEOLOGIA_Centro izquierda    0.000000
5           IDEOLOGIA_Derecha    0.000000
6         IDEOLOGIA_Izquierda    0.000000


In [15]:
# Guardar el modelo para uso futuro
import pickle
with open('./out/exp7_nacional.pkl', 'wb') as f:
    pickle.dump(modelo_exp_7_nacional, f)